In [15]:
!pip install Word2Vec

In [17]:
import Word2Vec
sentences = [['merkel', 'imposed', 'lockdown', 'germany'], ['chancellor', 'imposed', 'lockdown', 'germany'],
             ['paris', 'city', 'love']]
# Train the word2vec model
model = Word2Vec(sentences, size=2, window=4, sg=1, min_count=1, iter=8000)

# Create a df from the vectors of the all the words in the model
vectors = [model.wv[word] for word in model.wv.vocab.keys()]
df = pd.DataFrame(vectors, columns=['x', 'y'], index=model.wv.vocab.keys())
# Plot
fig = px.scatter(df, x="x", y="y", text=df.index)
fig.update_traces(textposition='top center')

ModuleNotFoundError: No module named 'Word2Vec'

In [2]:
!pip install nltk
!pip install wordcloud
!pip install gensim

     |████████████████████████████████| 220 kB 6.1 MB/s eta 0:00:01
  Created wheel for wordcloud: filename=wordcloud-1.8.1-cp38-cp38-macosx_10_9_x86_64.whl size=158427 sha256=e3d4bb59910b14aef5300cb757738e06d4582dcd91a9d2c528732cdb0411d9b2
  Stored in directory: /Users/shreyas.donki/Library/Caches/pip/wheels/4d/3f/0d/a2ba9b7895c9f1be89018b3141c3df3d4f9c786c882ccfbc3b
Successfully built wordcloud


In [3]:
import numpy as np 
import pandas as pd 
import nltk
from nltk.corpus import stopwords
import gensim
import string
import os

In [2]:
os.chdir(r"C:\Users\Raghavendra N\OneDrive\Official\Datasets")

In [18]:
df = pd.read_csv('imdb_sentiment.csv')

### Load Google's Pretrained model
- Download: https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit
- Size: 1.5GB

In [6]:
google_model = 'GoogleNews-vectors-negative300.bin.gz' 
embeddings = gensim.models.KeyedVectors.load_word2vec_format(google_model,
                                                        binary=True, limit = 500000)

In [19]:
embeddings['google'].shape

(300,)

In [20]:
embeddings.most_similar('computer')

[('computers', 0.7979379892349243),
 ('laptop', 0.6640493273735046),
 ('laptop_computer', 0.6548868417739868),
 ('Computer', 0.6473334431648254),
 ('laptop_computers', 0.5585449934005737),
 ('PC', 0.5539618134498596),
 ('laptops', 0.551753044128418),
 ('PCs', 0.5517113208770752),
 ('Computers', 0.5511513948440552),
 ('software', 0.5444108843803406)]

In [21]:
df.head(5)

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [22]:
df.shape

(748, 2)

In [23]:
df.columns

Index(['review', 'sentiment'], dtype='object')

In [24]:
df.drop_duplicates(inplace = True)

In [25]:
df.shape

(745, 2)

In [26]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [27]:
from string import punctuation
from nltk.corpus import stopwords
from nltk import word_tokenize
import re

In [28]:
import re
def clean_txt(sent):
    tokens = word_tokenize(re.sub('[^a-z ]',' ',sent.lower()))
    stop_updated = stopwords.words("english") + list(punctuation) +\
    ["..."] + \
    ["would", "could","told","subject"]
    text = [term for term in tokens if term not in stop_updated and
            len(term) > 2] 
    res = " ".join(text)
    return res

In [43]:
df['clean_review'] = df.review.apply(clean_txt)

In [44]:
df.head()

,review,sentiment,clean_review
0,"A very, very, very slow-moving, aimless movie ...",0,slow moving aimless movie distressed drifting ...
1,Not sure who was more lost - the flat characte...,0,sure lost flat characters audience nearly half...
2,Attempting artiness with black & white and cle...,0,attempting artiness black white clever camera ...
3,Very little music or anything to speak of.,0,little music anything speak
4,The best scene in the movie was when Gerardo i...,1,best scene movie gerardo trying find song keep...


## Create document vectors using word embeddings

In [31]:
docs_vectors = pd.DataFrame()
for doc in df.clean_review:
    temp = pd.DataFrame()
    words = doc.split(' ')
    for word in words:
        try:
            word2vec = embeddings[word]
            temp = temp.append(pd.Series(word2vec),ignore_index=True)
        except:
            pass
    doc_vector = temp.mean()
    docs_vectors = docs_vectors.append(doc_vector,ignore_index=True)

In [52]:
df.reset_index(drop=True)

,review,sentiment,clean_review
0,"A very, very, very slow-moving, aimless movie ...",0,slow moving aimless movie distressed drifting ...
1,Not sure who was more lost - the flat characte...,0,sure lost flat characters audience nearly half...
2,Attempting artiness with black & white and cle...,0,attempting artiness black white clever camera ...
3,Very little music or anything to speak of.,0,little music anything speak
4,The best scene in the movie was when Gerardo i...,1,best scene movie gerardo trying find song keep...
...,...,...,...
740,I just got bored watching Jessice Lange take h...,0,got bored watching jessice lange take clothes
741,"Unfortunately, any virtue in this film's produ...",0,unfortunately virtue film production work lost...
742,"In a word, it is embarrassing.",0,word embarrassing
743,Exceptionally bad!,0,exceptionally bad


In [32]:
docs_vectors.head()

,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,0.158325,0.205505,-0.087540,0.064842,-0.106415,0.008987,-0.062489,0.000954,0.057213,0.050742,...,-0.106262,0.061035,-0.097076,-0.020645,-0.056183,0.068665,0.088745,0.022263,0.062592,0.040619
1,0.143494,-0.017984,-0.009964,0.106308,-0.052292,-0.075745,-0.015942,-0.037323,0.114197,0.029335,...,-0.080658,0.051147,-0.152344,-0.048691,-0.065536,-0.029953,0.032959,-0.053650,0.021364,-0.096176
2,0.100850,0.048157,0.005754,0.006382,-0.103702,-0.006626,0.047667,0.038049,0.043840,0.020518,...,-0.120449,0.075370,-0.120959,0.036933,-0.079654,-0.011673,0.037342,-0.039069,0.012308,0.010465
3,0.070648,-0.059143,-0.003357,0.084737,-0.092316,0.107788,0.156372,-0.170471,0.006287,-0.007477,...,-0.052734,0.080322,-0.023529,0.030060,-0.011902,-0.059326,0.144897,-0.071167,0.074524,-0.052612
4,0.120809,0.007568,0.040127,0.048381,-0.056264,0.004886,0.024143,-0.053096,0.066806,-0.012214,...,-0.098731,0.089857,-0.140744,0.013428,-0.042019,-0.015290,0.079617,0.000027,0.037869,0.076067


In [33]:
docs_vectors = docs_vectors.fillna(0)

In [34]:
docs_vectors.shape

(745, 300)

In [ ]:
X=np.

In [35]:
X = docs_vectors
y = df.sentiment.values

In [36]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state=100)

In [37]:
from sklearn.linear_model import LogisticRegression 
classifier = LogisticRegression()
classifier.fit(X_train,y_train)

LogisticRegression()

In [38]:
y_pred = classifier.predict(X_test)

In [39]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [40]:
accuracy_score(y_test,y_pred)

0.8526785714285714

In [41]:
confusion_matrix(y_test,y_pred)

array([[102,  17],
       [ 16,  89]])